# Video compression

According to the dataset paper, 

> However, for each target video in the test set, we randomly selected two clips out of three and applied augmentations that
approximate actual degradations seen in real-life video distributions. Specifically, these augmentations were
> 1. reduce the FPS of the video to 15
> 2. reduce the resolution of the video to 1/4 of its original size
> 3. reduce the overall encoding quality

In this notebook we look at how to compress videos using `ffmpeg`.  Based on the above list, a quick search returns several useful examples:

* General compression for the web 1: https://dev.to/benjaminblack/use-ffmpeg-to-compress-and-convert-videos-458l  
* General compression for the web 2: https://trac.ffmpeg.org/wiki/Encode/YouTube   
* Frame rate change: https://trac.ffmpeg.org/wiki/ChangingFrameRate  
* Scaling: https://trac.ffmpeg.org/wiki/Scaling  

The latest `ffmpeg` can be obtained at https://johnvansickle.com/ffmpeg/.  See the FAQ for how to install.

In [ ]:
#default_exp video_compress

In [ ]:
#export
from IPython.display import HTML
import cv2
from fastai.core import *
from fastai.vision import *
from kgl_deepfake.data import *

In [ ]:
SOURCE = Path('../data/dfdc_train_part_0/')

In [ ]:
fnames = get_files(SOURCE, extensions=['.mp4'])

In [ ]:
fn = fnames[159]
fn

PosixPath('../data/dfdc_train_part_0/bfktkdugru.mp4')

In [ ]:
HTML(html_vid(fn))

Use `mediainfo` to show video properties

In [ ]:
! mediainfo {fn}

General
Complete name                            : ../data/dfdc_train_part_0/bfktkdugru.mp4
Format                                   : MPEG-4
Format profile                           : Base Media
Codec ID                                 : isom (isom/iso2/avc1/mp41)
File size                                : 15.2 MiB
Duration                                 : 10 s 24 ms
Overall bit rate                         : 12.7 Mb/s
Writing application                      : Lavf57.71.100

Video
ID                                       : 1
Format                                   : AVC
Format/Info                              : Advanced Video Codec
Format profile                           : High@L4
Format settings                          : CABAC / 4 Ref Frames
Format settings, CABAC                   : Yes
Format settings, Reference frames        : 4 frames
Codec ID                                 : avc1
Codec ID/Info                            : Advanced Video Coding
Duration                    

### File size

To conveniently check the file size of compressed videos:

In [ ]:
#export
def get_file_size(fpath):
    assert fpath.is_file()
    p = subprocess.run(f"du -hs {fpath}".split(), stdout=subprocess.PIPE)
    return p.stdout.decode().split('\t')[0]

In [ ]:
#export
Path.file_size = get_file_size

Original file size

In [ ]:
fn.file_size()

' 15M'

### Decorator to run command in terminal

In [ ]:
#export
def runnit(f):
    def _func(*args, **kwargs):
        command = f(*args, **kwargs)
        p = subprocess.run(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return p
    return _func

### Are these videos HDR?

The following checks whether the video is HDR or not.

In [ ]:
#export
@runnit
def vid_colour_primaries(fpath):
    "Use `mediainfo` to get video's colour primaries."
    return f'''
    mediainfo f'{fpath}' --Inform="Video;%colour_primaries%"
    '''

def vid_is_HDR(fpath):
    "Is video HDR or not?"
    p = vid_colour_primaries(fpath)
    res = p.stdout.decode()
    return True if 'BT.2020' in res else False

In [ ]:
vid_is_HDR(fn)

False

### Show video properties using `mediainfo`

In [ ]:
#export
def get_mediainfo(fpath=None): 
    @runnit
    def _func(fpath=None):
        return f'''mediainfo {fpath}'''
    p = _func(fpath=fpath)
    return p.stdout.decode()

### Encoding

Each of the URLs above does something different: encoding, scaling, fps.  These are combined into a single function.  Note that probably not all combinations of input parameters will run, but the main parameters to adjust are:

* **crf** "The range of the CRF scale is 0–51, where 0 is lossless, 23 is the default, and 51 is worst quality possible. A lower value generally leads to higher quality, and a subjectively sane range is 17–28. Consider 17 or 18 to be visually lossless or nearly so; it should look the same or nearly the same as the input but it isn't technically lossless." > 23 recommended.
* **fps** "Changing frame rates requires the video to be re-encoded. Without setting appropriate output quality or bit rate, the video quality may be degraded. Please look at the respective encoding guides for the codec you've chosen."
* **scale** Value between 0 and 1 to scale the original lengths.

To display the video and its properties after it's been compressed

In [ ]:
#export
def show_vid_info(f):
    "Display video and its properties after it's been processed."
    def _f(*args, **kwargs):
        p = f(*args, **kwargs)
        fpath = Path(kwargs['fpath_to'])
        display(HTML(html_vid(fpath)))
        print(get_mediainfo(fpath))
    return _f

Function for donig the encoding using `ffmpeg`, with default arguments defined.

In [ ]:
#export
def _ffmpeg_web_defaults():
    '''
    These are some reasonable values for uploading. i.e. YouTube, etc.
    '''
    return dict(video_encoder='libx264', video_bitrate='1.5M', fps=30, scale=.5, crf=23, #17-28
                audio_encoder='aac', audio_bitrate='128k')

def _ffmpeg_defaults():
    return dict(video_encoder=None, video_bitrate=None, fps=None, scale=None, crf=None,
                audio_encoder=None, audio_bitrate=None)

def _ffmpeg_fmts():
    "ffmpeg options syntax"
    return dict(video_encoder='-c:v {video_encoder:s}', 
                video_bitrate='-b:v {video_bitrate:s}', 
                fps='-r {fps:d}', 
                scale='-vf scale=iw*{scale:.2f}:ih*{scale:.2f}', 
                crf='-crf {crf:d}',
                audio_bitrate='-b:a {audio_bitrate}', 
                audio_encoder='-c:a {audio_encoder}')

#@show_vid_info
@runnit
def run_ffmpeg(fpath_from=None, fpath_to=None, **kwargs):
    '''
    Run ffmpeg
    '''
    ps = _ffmpeg_defaults()
    ps.update(kwargs)
    pstr = []
    for n, s in _ffmpeg_fmts().items():
        if ps[n] is None: pstr.append('')
        else: pstr.append(s.format(**ps))
    pstr = ' '.join(pstr)
    return f'ffmpeg -i {fpath_from} {pstr} {fpath_to}'

Apply to the original (uncomment the `@show_vid_info` above `run_ffmpeg` to display legible output in the next cell):

In [ ]:
f = Path(f'enc_{fn.name}')
if f.exists(): os.remove(f)
run_ffmpeg(fpath_from=fn, fpath_to=f, crf=28)

CompletedProcess(args=['ffmpeg', '-i', '../data/dfdc_train_part_0/bfktkdugru.mp4', '-crf', '28', 'enc_bfktkdugru.mp4'], returncode=0, stdout=b'', stderr=b"ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers\n  built with Apple clang version 11.0.0 (clang-1100.0.33.16)\n  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.2.2_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags='-I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include -I/Library/Java/JavaVirtualMachines/adoptopenjdk-13.0.1.jdk/Contents/Home/include/darwin -fno-stack-check' --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libmp3lame --enable-libopus --enable-librubberband --enable-libsnappy --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-lzma --enable-libfont

Double check the properties using `cv2`

In [ ]:
vcap = cv2.VideoCapture(str(f))
vlen = int(vcap.get(cv2.CAP_PROP_FRAME_COUNT))
ret, frame = vcap.read()
vcap.release()
vlen, frame.shape

(300, (1920, 1080, 3))

## Generate compressed sets

Use this section to generate sets of compressed videos.  To keep things simple to begin with, for each directory of original videos, generate 3 compressed sets: one at `fps=15`, one at quarter the size, `scale=.5`, and one with poorer encoding quality, `crf=28`.

In [ ]:
SOURCE = Path('../data')

In [ ]:
DFDC = Path('dfdc_train_part_0')

In [ ]:
ENC_DICT = dict(fps15={'fps':15}, quartersize={'scale':.5}, crf28={'crf':28})

In [ ]:
#export
def origfake_exist(row):
    fake_exists = row.fname.exists() 
    orig_exists = True if pd.isna(row.original) else row.original.exists()
    return (fake_exists and orig_exists)

def load_dfdc_json(path, dfdc, fpath=False, drop_missing=False, fill_real_na=False):
    a = get_files(path/dfdc, extensions=['.json'])[0]
    a = pd.read_json(a).T
    a.reset_index(inplace=True)
    a.rename({'index':'fname'}, axis=1, inplace=True)
    if fpath or drop_missing:
        a.fname = a.fname.apply(lambda o: path/dfdc/o)
        a.original = a.original.apply(lambda o: np.nan if pd.isna(o) else path/dfdc/o)
    if drop_missing: 
        exists = a.apply(origfake_exist, axis=1)
        a = a[exists].reset_index(drop=True)
    if fill_real_na:
        ao = a[a.label=='REAL'].copy()
        ao.drop('original', axis=1, inplace=True)
        ao['original'] = ao.fname.copy()
        a[a.label=='REAL'] = ao        
    return a

In [ ]:
vant = load_dfdc_json(SOURCE, DFDC, fpath=True, drop_missing=True,
                      fill_real_na=True)

In [ ]:
fnos = vant.original.unique()
fnos = fnos.astype(str)
fnos.sort()
fnos = np.array([Path(o) for o in fnos])

In [ ]:
# Run this to reduce number of original videos,
# if disk space is an issue
#idxs = torch.randperm(len(fnos))
#idxs = idxs[:int(.1 * len(idxs))]
fnos = fnos[:10]

In [ ]:
#export
def get_cvant_paper(vant:pd.DataFrame, fnos:list):
    '''
    Get compressed video annotation, `cvant`, according to 
    the DFDC paper.
    vant: video annotation
    fnos: iterable of original video names
    '''
    encs = list(ENC_DICT.keys()) + ['None']
    idxs = torch.randperm(len(fnos))
    chidxs = torch.chunk(idxs, len(encs))
    vantps = []
    for enc, idxs in zip(encs, chidxs):
        vantp = vant.set_index('original').loc[fnos[idxs]]
        vantp['enc'] = enc
        vantp = vantp.reset_index()
        vantps.append(vantp)
    cvant = pd.concat(vantps)
    return cvant

In [ ]:
cvant = get_cvant_paper(vant, fnos)

Define directory path to save the compressed videos.

In [ ]:
dir_out = SOURCE/f'{DFDC}_cv'
dir_out

PosixPath('../data/dfdc_train_part_0_cv')

In [ ]:
#export
def compress_vids(cvant, dir_out='compressed_videos'):
    '''
    Compress videos, save them and the annotation.
    '''
    dir_out = Path(dir_out)
    os.makedirs(dir_out, exist_ok=True)
    cvant['fname_cv'] = cvant.fname.apply(lambda o: dir_out/o.name)
    pb = progress_bar(list(cvant.iterrows()))
    for _, r in pb:
        if r.enc=='None': 
            shutil.copy(r.fname, r.fname_cv)
        else:
            run_ffmpeg(fpath_from=r.fname, fpath_to=r.fname_cv, **ENC_DICT[r.enc])
        pb.comment = f'{r.fname.name} {r.enc}'
    cvant.to_csv(dir_out/'metadata.csv', index=False)
    return cvant

**Remember to comment out the `@show_vid_info` to avoid lots of output printed in the next cell.**

In [ ]:
cvant = compress_vids(cvant, dir_out)

In [ ]:
cvant.shape

(105, 6)

# -fin

In [ ]:
from nbdev.export import *

In [ ]:
notebook2script()

Converted 00_data.ipynb.
Converted 00a_video_compression.ipynb.
Converted 01_face_detection.ipynb.
Converted 01a_faces_probs_examples.ipynb.
Converted 01a_faces_probs_examples_hv.ipynb.
Converted 02_fix_luminosity.ipynb.
Converted 02a_create_faceimage_dataset.ipynb.
Converted 02bis_Create_Dataset.ipynb.
Converted 02c_faces_different_dfdc_zips.ipynb.
Converted 03_models.ipynb.
Converted 04_Baseline_Classification.ipynb.
Converted 04_Classification.ipynb.
Converted 04a_classification_videolist.ipynb.
Converted 05_Class_Imbalance.ipynb.
Converted 06_Focal_Loss.ipynb.
Converted 07_full_classification.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 07a_classify_video_margin.ipynb.
Converted 07b_classify_resize.ipynb.
Converted 08_Validation.ipynb.
Converted 09_Data_Augmentation.ipynb.
Converted deepfake_submission.ipynb.
Converted export_kernel_module.ipynb.
Converted fake_original.ipynb.
Converted organising_face_detection_results.ipynb.
Converted test_subm